In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
DIR_IN_QC="../../output/nucleotideFreq_QCreads/"
samples_metadata <- read.csv(file = "../../output/samples_metadata.csv") %>% as.data.table()

list.files(DIR_IN_QC, pattern = "_endMotif_count.txt") %>% length()

In [ ]:
if (exists("QCreads_meta")){rm("QCreads_meta")}
profvis({
for (file in list.files(DIR_IN_QC, pattern = "_endMotif_count.txt")){
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    side_tmp <- unlist(strsplit(file, "_"))[2]
    
    if(countLines(paste0(DIR_IN_QC, file)) > 1){
        data_tmp <- fread(paste0(DIR_IN_QC, file), sep = ",") %>% 
            mutate(read = side_tmp) %>% 
            mutate(sample_id = sample_tmp) %>% 
            filter(!grepl(EndMotif, pattern = "N")) %>% 
            select(c("sample_id", "read", "EndMotif", "Count")) 
        
        tmp_IS_meta <- merge(x = samples_metadata, y = data_tmp, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("QCreads_meta")){QCreads_meta <- tmp_IS_meta} else {
            QCreads_meta <- rbind(QCreads_meta, tmp_IS_meta)}
    }
    fwrite(QCreads_meta, "../../output/PP_figures/QCreads_EndMotifs.csv", row.names = FALSE)
}
})